### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches

In [ ]:
test_percentage = 0.4

In [ ]:
loss_str = "bce"
with_augmentation = False
stride=512
seed = 42
run_id = 1

aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
root_path = f"/content/datasets/hrf_512_{stride}_{loss_str}{aug_string}_{run_id}/"
images_path = f"{root_path}/images/"
gt_path = f"{root_path}/groundtruth"

create_dir(images_path)
create_dir(gt_path)

In [ ]:
root_path

'/content/datasets/hrf_512_512_bce_1/'

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/images/*.* {images_path}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/HRF/groundtruth/*.* {gt_path}



In [ ]:
images_files, gt_files = get_filenames_sorted(images_path, gt_path)
print(len(images_files), len(gt_files))
# train_test_split
from sklearn.model_selection import train_test_split
label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=test_percentage, random_state=seed)

45 45


In [ ]:
print(len(xtrain), len(xtest))
print(label)

27 18
['dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h', 'dr', 'g', 'h']


In [ ]:
from fundal_image.utils import create_dir
# from patches import generate_patches_images_only

In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (512, 512)}

dataset_name = "HRF"
num_pixels_to_add = [0, 0]
if (image_sizes["HRF"][0] % patch_size["HRF"][0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["HRF"][0] // patch_size["HRF"][0]) + 1) * patch_size["HRF"][0]
    num_pixels_to_add[1] = ((image_sizes["HRF"][1] // patch_size["HRF"][1]) + 1) * patch_size["HRF"][1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_512_{stride}_{split}.csv", index=False)



Generating patches for 27 images


100%|██████████| 27/27 [00:18<00:00,  1.48it/s]


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size["HRF"],
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_512_{stride}_{split}.csv", index=False)


Generating patches for 18 images


100%|██████████| 18/18 [00:11<00:00,  1.53it/s]


In [ ]:
metadata_test_df.shape

(630, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/hrf_512_512_bce_1//images/13...,0,0,512,512,512,103562
1,/content/datasets/hrf_512_512_bce_1//images/13...,0,512,512,512,512,199885
2,/content/datasets/hrf_512_512_bce_1//images/13...,0,1024,512,512,512,204800
3,/content/datasets/hrf_512_512_bce_1//images/13...,0,1536,512,512,512,204800
4,/content/datasets/hrf_512_512_bce_1//images/13...,0,2048,512,512,512,204800


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['11_dr', '05_dr', '07_dr', '01_h', '15_g', '06_h', '15_h', '04_dr', '10_h', '06_g', '02_g', '05_g', '13_dr', '09_h', '03_g', '08_dr', '08_g', '12_h']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

def bce(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets)

    return Dice_BCE



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
else:
    raise Exception("Invalid value for loss_str")
    


In [ ]:
loss

<function __main__.bce>

In [ ]:
H = 512
W = 512
loss = dice_loss

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
model_dir = f"{root_path}/trained_models/"

In [ ]:
create_dir(model_dir)

In [ ]:
!ls {model_dir}

In [ ]:
%cd /content/datasets

/content/datasets


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 8
lr = 1e-4
num_epochs = 50
model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 945 - 945
Valid: 630 - 630
Epoch 1/50
119/119 [==============================] - ETA: 0s - loss: 0.6012 - dice_coef: 0.3974 - iou: 0.2564 - recall: 0.8023 - precision: 0.3233 - auc: 0.9099
Epoch 00001: val_loss improved from inf to 0.89584, saving model to /content/datasets/hrf_512_512_bce_1//trained_models/model.h5
119/119 [==============================] - 153s 1s/step - loss: 0.6012 - dice_coef: 0.3974 - iou: 0.2564 - recall: 0.8023 - precision: 0.3233 - auc: 0.9099 - val_loss: 0.8958 - val_dice_coef: 0.1042 - val_iou: 0.0551 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_auc: 0.5032 - lr: 1.0000e-04
Epoch 2/50
119/119 [==============================] - ETA: 0s - loss: 0.4231 - dice_coef: 0.5745 - iou: 0.4059 - recall: 0.8159 - precision: 0.6770 - auc: 0.9466
Epoch 00002: val_loss did not improve from 0.89584
119/119 [==============================] - 129s 1s/step - loss: 0.4231 - dice_coef: 0.5745 - iou: 0.4059 - recall: 0.8159 - precision: 0.6770 - auc: 0.9466 -

### Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:
H = 512
W = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/hrf_512_512_bce_1//results/'

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
#    print(ori_x.shape, ori_y.shape, y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef"])
df.to_csv(f"{results_path}/score_full.csv")

 11%|█         | 70/630 [00:33<04:22,  2.13it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 14%|█▍        | 91/630 [00:43<04:18,  2.09it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr

Accuracy: 0.97474
F1: 0.75144
Jaccard: 0.62238
Recall: 0.73026
Precision: 0.79878
AUC: 0.88815
Dice Coeff: 0.73854


# Reconstruct original image

In [ ]:
prediction_np_path

'/content/datasets/hrf_512_512_bce_1//predictions_np/'

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

file /content/datasets/hrf_512_512_bce_1//predictions_np/_06_g_33.npy
06_g 33
file /content/datasets/hrf_512_512_bce_1//predictions_np/_07_dr_25.npy
07_dr 25
file /content/datasets/hrf_512_512_bce_1//predictions_np/_08_dr_16.npy
08_dr 16
file /content/datasets/hrf_512_512_bce_1//predictions_np/_11_dr_9.npy
11_dr 9
file /content/datasets/hrf_512_512_bce_1//predictions_np/_07_dr_26.npy
07_dr 26
file /content/datasets/hrf_512_512_bce_1//predictions_np/_15_h_1.npy
15_h 1
file /content/datasets/hrf_512_512_bce_1//predictions_np/_15_g_32.npy
15_g 32
file /content/datasets/hrf_512_512_bce_1//predictions_np/_01_h_0.npy
01_h 0
file /content/datasets/hrf_512_512_bce_1//predictions_np/_13_dr_6.npy
13_dr 6
file /content/datasets/hrf_512_512_bce_1//predictions_np/_15_g_13.npy
15_g 13
file /content/datasets/hrf_512_512_bce_1//predictions_np/_09_h_12.npy
09_h 12
file /content/datasets/hrf_512_512_bce_1//predictions_np/_05_g_17.npy
05_g 17
file /content/datasets/hrf_512_512_bce_1//predictions_np/_02_g

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((2560, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)
    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:15<04:25, 15.61s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:30<04:07, 15.45s/it]

...

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:21<00:15, 15.42s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:37<00:00, 15.41s/it]

Accuracy: 0.97474
F1: 0.80703
Jaccard: 0.67759
Recall: 0.78840
Precision: 0.83365
AUC: 0.92166
Dice Coeff: 0.80664


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.6
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
#    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:15<04:25, 15.62s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:31<04:09, 15.58s/it]

...

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:21<00:15, 15.36s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:36<00:00, 15.38s/it]

Accuracy: 0.97476
F1: 0.80663
Jaccard: 0.67699
Recall: 0.78560
Precision: 0.83596
AUC: 0.92166
Dice Coeff: 0.80664


In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")
model = load_model(model_path, compile=False)

""" Load the dataset """
# dataset_path = os.path.join(f"{data_path}/{split_name}", "test")

test_x, test_y = load_data(f"{data_path}/{split_name}/reconstructed/test/")


""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)
    print(name)

#     """ Prediction """
    #y_pred = model.predict(np.expand_dims(x, axis=0))[0]

    orig_y_pred, y_pred = read_mask(f"{data_path}/{split_name}/reconstructed/test/predictions_reconstructed/{name}.png")
    print(y_pred.shape, np.max(y_pred), np.min(y_pred))
    print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.4
    y_pred = y_pred.astype(np.int32)

    """ Saving the images """
    save_image_path = f"{results_path_reconstructed}/{name}.png"
    print(ori_x.shape, ori_y.shape,y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)

    
    # cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice: {score[6]:0.5f}")

""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef"])
df.to_csv(f"{results_path_reconstructed}/score_full.csv")

  0%|          | 0/18 [00:00<?, ?it/s]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/01_h.png
01_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/01_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//01_h.png


  6%|▌         | 1/18 [00:15<04:25, 15.60s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/02_g.png
02_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/02_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//02_g.png


 11%|█         | 2/18 [00:31<04:12, 15.76s/it]

...

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_g.png
15_g
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_g.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_g.png


 94%|█████████▍| 17/18 [04:25<00:15, 15.63s/it]

/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/mask/15_h.png
15_h
/content/datasets/hrf_512_512_bce_1//split_2/reconstructed/test/predictions_reconstructed/15_h.png
(2560, 3584) 1.0 0.0
(2560, 3584) 255 0
(2560, 3584, 3) (2560, 3584) (2560, 3584)
Saving image of size (2560, 10772, 3) to /content/datasets/hrf_512_512_bce_1//results_reconstructed//15_h.png


100%|██████████| 18/18 [04:41<00:00, 15.63s/it]

Accuracy: 0.97472
F1: 0.80740
Jaccard: 0.67813
Recall: 0.79108
Precision: 0.83143
AUC: 0.92166
Dice: 0.80664


In [ ]:
results_path_reconstructed

'/content/datasets/hrf_512_512_bce_1//results_reconstructed/'

In [ ]:
!ls 

hrf_512_512_bce_1  logs


In [ ]:
!zip -r hrf_512_512_bce_1.zip  hrf_512_512_bce_1/

Streaming output truncated to the last 5000 lines.
  adding: hrf_512_512_bce_1/split_2/train/image/_13_g_33.png (deflated 3%)
  adding: hrf_512_512_bce_1/split_2/train/image/_02_dr_30.png (deflated 4%)
  adding: hrf_512_512_bce_1/split_2/train/image/_09_dr_1.png (deflated 3%)
  adding: hrf_512_512_bce_1/split_2/train/image/_12_dr_19.png (deflated 3%)
  adding: hrf_512_512_bce_1/split_2/train/image/_03_h_2.png (deflated 5%)
  adding: hrf_512_512_bce_1/split_2/train/image/_03_dr_20.png (deflated 3%)
  adding: hrf_512_512_bce_1/split_2/train/image/_14_dr_26.png (deflated 3%)
  adding: hrf_512_512_bce_1/split_2/train/image/_13_g_32.png (deflated 4%)
  adding: hrf_512_512_bce_1/split_2/train/image/_01_g_18.png (deflated 5%)
  adding: hrf_512_512_bce_1/split_2/train/image/_10_g_9.png (deflated 5%)
  adding: hrf_512_512_bce_1/split_2/train/image/_03_dr_34.png (deflated 7%)
  adding: hrf_512_512_bce_1/split_2/train/image/_12_dr_30.png (deflated 4%)
  adding: hrf_512_512_bce_1/split_2/train/ima

In [ ]:
!ls -ltrh


total 1.6G
drwxr-xr-x  4 root root 4.0K Feb  3 14:45 logs
drwxr-xr-x 10 root root 4.0K Feb  3 16:05 hrf_512_512_bce_1
-rw-r--r--  1 root root 1.6G Feb  3 16:36 hrf_512_512_bce_1.zip


In [ ]:
!cp hrf_512_512_bce_1.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
%cd /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches


In [ ]:
!pwd

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches


In [ ]:
!ls

02_g_test.png
hrf_512_256_bce_hvr_xval_10_42.zip
hrf_512_512
hrf_512_512_bce
hrf_512_512_bce_1.zip
hrf_512_512_bce_2
hrf_512_512_bce_2.zip
hrf_512_512_bce_hvr
hrf_512_512_bce_hvr_0_3
hrf_512_512_bce_hvr_0_33_42.zip
hrf_512_512_bce_hvr_0_33_43
hrf_512_512_bce_hvr_0_33_43.zip
hrf_512_512_bce_hvr_0_3_44
hrf_512_512_bce_hvr_0_3_44.zip
hrf_512_512_bce_hvr_feb_3_1.zip
hrf_512_512_bce_hvr_xval_2_42.zip
hrf_512_512_bce_hvr_xval_5_42
hrf_512_512_bce_hvr_xval_5_42_10epochs
hrf_512_512_bce_hvr_xval_5_42.csv
hrf_512_512_bce_hvr.zip
hrf_512_512_bce.zip
hrf_512_512_dice
hrf_512_512_dice_bce
hrf_512_512_dice_bce_hvr
hrf_512_512_dice_bce_hvr_2.zip
hrf_512_512_dice_bce_hvr_jan_31_2.zip
hrf_512_512_dice_bce_hvr_jan_31.zip
hrf_512_512_dice_bce_hvr_xval_5_42.zip
hrf_512_512_dice_bce_hvr_xval_9_42_10epochs.zip
hrf_512_512_dice_bce_hvr.zip
hrf_512_512_DiceBCELoss
hrf_512_512_dice_loss_2
hrf_512_512_dice_loss_2.zip
hrf_512_512_dice_loss_hvr_0_3_feb_3.zip
hrf_512_512_dice_loss_hvr_xval_5_42.zip
hrf_512_512_di

In [ ]:
!unzip hrf_512_512_bce_1.zip


Streaming output truncated to the last 5000 lines.
  inflating: hrf_512_512_bce_1/split_2/train/image/_13_g_33.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_02_dr_30.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_09_dr_1.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_12_dr_19.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_03_h_2.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_03_dr_20.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_14_dr_26.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_13_g_32.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_01_g_18.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_10_g_9.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_03_dr_34.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_12_dr_30.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_10_g_33.png  
  inflating: hrf_512_512_bce_1/split_2/train/image/_15_dr_24.png  
  inflating: hrf_512

In [ ]:
drive.flush_and_unmount()